In [13]:
import numpy as np

import pandas as pd

import sys, os

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
     sys.path.append(module_path+"//utils")

from tqdm import tqdm

import semantic_term_placement

from collections import Counter


# Create Annoy Index 

* Create two indices


    * Context around each sample is generated by the "Tagger"
    * "top_terms" are pregenerated keywords for each sample, extracted from the the context around each sample using the "KeyBERT"
    * "label" for candidates will be same as existing LSF or non-LSF samples if they have a close neighbour from them otherwise candidates remain unlabeled (-1 for BERTopic)

In [4]:
df_context_all=pd.read_csv('../../data/Final_Context.tsv',sep='\t')

index_lsf_names=df_context_all[df_context_all.serial< 200000].name.tolist()
index_lsf_labels=df_context_all[df_context_all.serial< 200000].label.tolist()

index_non_lsf_names=df_context_all[(df_context_all.serial>= 200000) &  (df_context_all.serial< 300000)].name.tolist()
index_non_lsf_labels=df_context_all[(df_context_all.serial>= 200000) &  (df_context_all.serial< 300000)].label.tolist()


In [9]:
df_context_all.head()

,Unnamed: 0,serial,match_count,name,label,real_match_count,top_terms,ngrams
0,0,100000,5,surgery and interventions,0,57,coronary cardiothoracic surgery arteries inter...,surgery and interventions
1,1,100002,153234,Nutrition,1,922423,nutrition supplementation parenteral nutrition...,Parenteral Nutrition. Parenteral nutrition. pa...
2,2,100003,24361,Drugs,2,145723,druginteractionslabeling pharmacology pharmaco...,CERTAIN DRUGS. Certain Drugs. Drugs associated...
3,3,100004,6530,Environmental exposures,3,45927,toxicity carcinogens environmentalexposures po...,environmental exposures. Environmental Exposur...
4,4,100005,20,Food processing and preparation,1,120,preparation procedures processing methods tech...,Food processing and preparation


In [5]:

index_lsf=semantic_term_placement.build_annoy_index(index_lsf_names)
index_non_lsf=semantic_term_placement.build_annoy_index(index_non_lsf_names)

Building the index with 100 trees...
Index is successfully built.
Building the index with 100 trees...
Index is successfully built.


# Load Test Candidates

In [24]:
def load_test_data(file_path):
    test_candidates=pd.read_csv(file_path,sep='\t')
    test_labels=test_candidates.Label.to_numpy()
    test_candidates_context=test_candidates.top_terms.to_numpy()
    test_candidates_names=test_candidates.name.to_numpy()

    label_distribution=Counter(test_labels)
    print(label_distribution)
    return test_candidates_names,test_candidates_context,test_labels


def generate_binary_label(test_candidates):
    lsf_labels=list(range (9))
    # assign 1 to LSF samples and 0 to non-LSF samples
    binary_labels=[]
    lsf_indices=[]
    non_lsf_indices=[]
    for i,label in enumerate(test_candidates):
        if label in lsf_labels:
            lsf_indices.append(i)
            binary_labels.append(1)
        else:
            non_lsf_indices.append(i)
            binary_labels.append(0)
    return binary_labels



file_path='../../data/test/test.tsv'
test_candidates_names,test_candidates_context,test_labels=load_test_data(file_path)
binary_labels=generate_binary_label(test_labels)

Counter({-1: 110, 1: 15, 3: 9, 0: 3, 2: 3, 5: 2, 8: 2, 6: 1})


# Distinguish LSF using Annoy Index

In [18]:
def predict_by_KNN (candidates,index_lsf,index_lsf_names,index_non_lsf,index_non_lsf_names):
    probs_KNN=[]
    for i,name in enumerate(candidates):
        
        _,ditance_lsf=semantic_term_placement.find_neighbors(index_lsf,index_lsf_names, query_name =name,num_matches=3)
        _,distance_non_lsf=semantic_term_placement.find_neighbors(index_non_lsf,index_non_lsf_names,query_name=name,num_matches=3)

        score_lsf=1/np.mean(ditance_lsf)**2
  
        score_nonLSF=1/np.mean(distance_non_lsf)**2
      
        sum_scores=sum([score_lsf,score_nonLSF])
        probs_KNN.append(score_lsf/sum_scores)
    return probs_KNN

        

In [26]:
probs_KNN=predict_by_KNN(test_candidates,index_lsf,index_lsf_names,index_non_lsf,index_non_lsf_names)
